In [193]:
import numpy as np
import cmath
import math
from sklearn.preprocessing import normalize
from scipy.constants import epsilon_0 as ε_0, c, pi as π, e, hbar as ℏ
eV = e
from scipy.special import lpmn, sph_jn, sph_yn
from scipy.misc import factorial
from numpy import newaxis as nx

I = np.identity(3)

## Green's functions
All vectors are expected to be in cartesian coordinates. Arguments and return values are in form of arrays ("lists") to enable fast batch processing.
### Free space

In [23]:
# Transverse delta; invalit at r1 = r2
def δ_T(r1, r2):
    print("Not implemented")
    return

# Longitudinal delta; invalid at r1 = r2
def δ_L(r1, r2):
    print("Not implemented")
    return

def G_fs(r1, r2, ω):
    r12 = np.linalg.norm((r1-r2), axis=1)
    g = np.exp(1j * r12 * ω/c) / (4*π*r12)
    return np.outer(g, I)    

def K_fs(r1, r2, ω):
    print("Not implemented")
    return

### Spherical particle (Mie solution)
TODO

In [233]:
# r[:,0] = x = r sin θ cos φ
# r[:,1] = y = r sin θ sin φ
# r[:,2] = z = r cos θ
def spherical_vector_waves(nmax, k, r):
    # FIXME the implementation is _wrong_ very near z axis
    # and the problem remains in the derivatives of Legendre polynomials
    # k is single scalar, r is single 3D vector
    r.shape = (3) # FIXME do proper check
    ρ = np.linalg.norm(r)
    r̂ = r/ρ
    cosθ = r̂[2]
    sinθ = np.sqrt(1-cosθ*cosθ) # FIXME np->math
    cosφ = r̂[0]/sinθ if sinθ else 0 # avoid zero division here
    sinφ = r̂[1]/sinθ if sinθ else 1
    φ = np.arctan2(sinφ,cosφ)
    # TODO can those be views so I don't have to write the indices
    # explicitly?
    m = np.arange(nmax+1)
    n = m
    sinmφ = np.sin(m*φ)
    cosmφ = np.cos(m*φ)
    θ̂ = np.array([cosθ*cosφ,cosθ*sinφ,-sinθ])
    φ̂ = np.array([-sinφ, cosφ, 0])
    jnkr2 = sph_jn(nmax, k*ρ)
    # scipy.special.lpmn is for real arguments (see also clpmn)
    # + derivatives
    # m's are non-negative, cf. Bohren&Huffman (4.23)
    Pmn2 = lpmn(nmax, nmax, cosθ)
    
    # FIXME this is _wrong_ very near z axis – but it avoids nan 
    # and should give correct result exactly on z axis
    mdsinθ = m/(sinθ if sinθ else 1)
    
    Memn = (-mdsinθ[:,nx] * jnkr2[0][nx,:] * 
         Pmn2[0] * sinmφ[:,nx])[:,:,nx] * θ̂ [nx,nx,:] + (
         + sinθ * jnkr2[0][nx,:] * Pmn2[1] * cosmφ[:,nx])[:,:,nx] * φ̂ [nx,nx,:]
    Momn = (mdsinθ[:,nx] * jnkr2[0][nx,:] * 
         Pmn2[0] * cosmφ[:,nx])[:,:,nx] * θ̂ [nx,nx,:] + (
         + sinθ * jnkr2[0][nx,:] * Pmn2[1] * sinmφ[:,nx])[:,:,nx] * φ̂ [nx,nx,:]
    Npfac = jnkr2[0]/(k*ρ) + jnkr2[1]
    Nemn = (((n*(n+1)/(k*ρ))*jnkr2[0])[nx,:]
         * Pmn2[0] * cosmφ[:,nx])[:,:,nx] * r̂[nx,nx,:] + ( Npfac[nx,:] *
         (-sinθ) * Pmn2[1] * cosmφ[:,nx])[:,:,nx] * θ̂ [nx,nx,:] + (
         Npfac[nx,:] * (-mdsinθ)[:,nx] * Pmn2[0] * sinmφ[:,nx])[:,:,nx] * φ̂ [nx,nx,:]
    Nomn = (((n*(n+1)/(k*ρ))*jnkr2[0])[nx,:]
         * Pmn2[0] * sinmφ[:,nx])[:,:,nx] * r̂[nx,nx,:] + ( Npfac[nx,:] *
         (-sinθ) * Pmn2[1] * sinmφ[:,nx])[:,:,nx] * θ̂ [nx,nx,:] + (
         Npfac[nx,:] * mdsinθ[:,nx] * Pmn2[0] * cosmφ[:,nx])[:,:,nx] * φ̂ [nx,nx,:]
    return (Memn, Momn, Nemn, Nomn)

#TODO: test against my old code (electroballz)!
def mie_reflection_coefficients(a, nmax, ε_m, ε_b, ω):
    # permittivities are relative!
    # cf. PRB 85, 075303, Appendix
    # assuming unit rel. permeability
    # for general permeability, cf. van Vlack's
    # dissertation, pp. 41–42
    k_m = cmath.sqrt(ε_m) * ω / c
    x_m = k_m * a
    k_b = cmath.sqrt(ε_b) * ω / c
    x_b = k_b * a
    jnka2_m = sph_jn(nmax, x_m)
    jnka2_b = sph_jn(nmax, x_b)
    ynka2_m = sph_yn(nmax, x_m)
    ynka2_b = sph_yn(nmax, x_b)
    τ_m = jnka2_m[0]
    τ_b = jnka2_b[0]
    κ_m = τ_m + 1j * ynka2_m[0]
    κ_b = τ_b + 1j * ynka2_b[0]
    dτ_m = τ_m / x_m + jnka2_m[1]
    dτ_b = τ_b / x_b + jnka2_b[1]
    dκ_m = κ_m / x_m + jnka2_m[1] + 1j * ynka2_m[1]
    dκ_b = κ_b / x_b + jnka2_b[1] + 1j * ynka2_b[1]
    RH = (k_m*dτ_m*τ_b - k_b*dτ_b*τ_m) / (k_m*dτ_m*κ_b - k_b*dκ_b*τ_m)
    RV = (k_m*τ_m*dτ_b - k_b*τ_b*dτ_m) / (k_m*τ_m*dκ_b - k_b*κ_b*dτ_m)
    return (RH, RV)
    
def mie_G_scatt(a, nmax, ε_m, ε_b, ω, r1, r2):
    # the same applies as for mie_reflection_coefficients
    (RH, RV) = mie_reflection_coefficients(a=a, nmax=nmax, ε_m=ε_m, ε_b=ε_b, ω=ω)
    G = mie_G_scatt_precalcd(a=a, nmax=nmax, ε_b=ε_b, ω=ω, RH=RH, RV=RV, r1=r1, r2=r2)
    return G

def mie_G_scatt_precalcd(a, nmax, ε_b, ω, RH, RV, r1, r2):
    k_b = cmath.sqrt(ε_b) * ω / c
    n = np.arange(nmax + 1)
    m = np.arange(nmax + 1)
    # avoid zero division:
    n[0] = 1 
    oneminusδn = np.ones(nmax+1)
    oneminusδn[0] = 0
    
    twominusδm = np.ones(nmax+1) * 2
    twominusδm[0] = 1
    prefactor = (-1j * k_b / (4 * π)) * twominusδm[:,nx] * (
      (2*n+1)/(n*(n+1)))[nx,:] * (factorial(n[nx,:]-m[:,nx])
      / factorial(n[nx,:]+m[:,nx])) * oneminusδn [nx,:]
    (Me1, Mo1, Ne1, No1) = spherical_vector_waves(nmax, k_b, r1)
    (Me2, Mo2, Ne2, No2) = spherical_vector_waves(nmax, k_b, r2)
    G_contribs = prefactor[:,:,nx,nx] * (
        RH[nx,:,nx,nx] 
        * (Me1[:,:,:,nx] 
           * Me2[:,:,nx,:]                
           + Mo1[:,:,:,nx] 
           * Mo2[:,:,nx,:])
       +RV[nx,:,nx,nx] * (Ne1[:,:,:,nx] * Ne2[:,:,nx,:]
                          + No1[:,:,:,nx] * No2[:,:,nx,:]) 
    )
    return np.sum(G_contribs, axis=(0,1))


## T-matrix

In [ ]:


def M(G, ω, R, μ): # as in PRA 70, 053823, eq. (37)
    #M = eye(N)
    
    

# Examples

# Playground

In [201]:
n = np.arange(6)
n[:,nx] * n[nx,:]

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5],
       [ 0,  2,  4,  6,  8, 10],
       [ 0,  3,  6,  9, 12, 15],
       [ 0,  4,  8, 12, 16, 20],
       [ 0,  5, 10, 15, 20, 25]])

In [19]:
r = np.array([[1,0,0],[2,2,2]])
R = np.array([[0,1,0],[0,1,4]])
np.linalg.norm((r-R), axis=1)
G_fs(r, R, 2e8)

array([[ 0.03302973+0.04555572j,  0.00000000+0.j        ,
         0.00000000+0.j        ,  0.00000000+0.j        ,
         0.03302973+0.04555572j,  0.00000000+0.j        ,
         0.00000000+0.j        ,  0.00000000+0.j        ,
         0.03302973+0.04555572j],
       [-0.01107202+0.02410456j, -0.00000000+0.j        ,
        -0.00000000+0.j        , -0.00000000+0.j        ,
        -0.01107202+0.02410456j, -0.00000000+0.j        ,
        -0.00000000+0.j        , -0.00000000+0.j        ,
        -0.01107202+0.02410456j]])

In [218]:
np.seterr(divide='raise')
spherical_vector_waves(5, 1, np.array([0,0.0001,1]))


(array([[[ -0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [ -3.01168679e-05,   0.00000000e+00,   0.00000000e+00],
         [ -1.86105155e-05,   0.00000000e+00,   0.00000000e+00],
         [ -5.40394869e-06,   0.00000000e+00,   0.00000000e+00],
         [ -1.01101580e-06,   0.00000000e+00,   0.00000000e+00],
         [ -1.38841737e-07,   0.00000000e+00,   0.00000000e+00]],
 
        [[ -0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [ -1.84412629e-17,   3.01168678e-01,  -3.01168679e-05],
         [ -1.13956540e-17,   1.86105156e-01,  -1.86105156e-05],
         [ -3.30896412e-18,   5.40394864e-02,  -5.40394866e-06],
         [ -6.19068603e-19,   1.01101580e-02,  -1.01101580e-06],
         [ -8.50160375e-20,   1.38841736e-03,  -1.38841736e-07]],
 
        [[  0.00000000e+00,  -0.00000000e+00,   0.00000000e+00],
         [  0.00000000e+00,  -0.00000000e+00,   0.00000000e+00],
         [ -3.72210312e-05,  -4.55826167e-21,   4.55826169e-25],
         [ -2.70197

In [231]:
a=50e-9
nmax=5
ε_m=1.1+0.1j
ε_b=1
ω=c/500e-9
(RH,RV)=mie_reflection_coefficients(a=a, nmax=nmax, ε_m=ε_m, ε_b=ε_b, ω=ω)
mie_G_scatt_precalcd(a=a, nmax=nmax, ε_b=ε_b, ω=ω, RH=RH, RV=RV, r1=np.array([60e-9,0,0]), r2=np.array([0,60e-9,0]))

array([[ -2.33809776e+00 -2.19315598e+00j,
          2.02411316e-06 +1.87437796e-06j,
          0.00000000e+00 +0.00000000e+00j],
       [  5.41975756e-06 +5.64566844e-06j,
         -2.33809776e+00 -2.19315598e+00j,
          0.00000000e+00 +0.00000000e+00j],
       [  0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
         -2.33472952e+00 -2.18999654e+00j]])

In [232]:
mie_G_scatt(a=a, nmax=nmax, ε_m=ε_m, ε_b=ε_b, ω=ω, r1=np.array([60e-9,0,0]), r2=np.array([0,60e-9,0]))
#(RH, RV) = mie_reflection_coefficients(a, nmax, ε_m, ε_b, ω)
#mie_G_scatt_precalcd(a, nmax, ε_b, ω, RH, RV, r1, r2)m

[[ -2.33809776e+00 -2.19315598e+00j   2.02411316e-06 +1.87437796e-06j
    0.00000000e+00 +0.00000000e+00j]
 [  5.41975756e-06 +5.64566844e-06j  -2.33809776e+00 -2.19315598e+00j
    0.00000000e+00 +0.00000000e+00j]
 [  0.00000000e+00 +0.00000000e+00j   0.00000000e+00 +0.00000000e+00j
   -2.33472952e+00 -2.18999654e+00j]]


array([[ -2.33809776e+00 -2.19315598e+00j,
          2.02411316e-06 +1.87437796e-06j,
          0.00000000e+00 +0.00000000e+00j],
       [  5.41975756e-06 +5.64566844e-06j,
         -2.33809776e+00 -2.19315598e+00j,
          0.00000000e+00 +0.00000000e+00j],
       [  0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
         -2.33472952e+00 -2.18999654e+00j]])

In [160]:
    # k is single scalar, r is single 3D vector
    r.shape = (3) # FIXME do proper check
    ρ = np.linalg.norm(r)
    r̂ = r/ρ
    cosθ = r̂[2]
    sinθ = np.sqrt(1-cosθ*cosθ)
    cosφ = r̂[0]/sinθ if sinθ else 0 # avoid zero division here
    sinφ = r̂[1]/sinθ if sinθ else 1
    φ = np.arctan2(sinφ,cosφ)
    # TODO can those be views so I don't have to write the indices
    # explicitly?
    m = np.arange(nmax+1)
    n = m
    sinmφ = np.sin(m*φ)
    cosmφ = np.cos(m*φ)
    θ̂ = np.array([cosθ*cosφ,cosθ*sinφ,-sinθ])
    φ̂ = np.array([-sinφ, cosφ, 0])
    jnkr2 = sph_jn(nmax, k*ρ)
    # scipy.special.lpmn is for real arguments (see also clpmn)
    # + derivatives
    # m's are non-negative, cf. Bohren&Huffman (4.23)
    Pmn2 = lpmn(nmax, nmax, cosθ)
    mdsinθ = m/(sinθ if sinθ else 1)
    Npfac = jnkr2[0]/(k*ρ) + jnkr2[1]

    Pmn2


(array([[ 1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[   0.,    1.,    3.,    6.,   10.,   15.],
        [   0.,   inf,   inf,   inf,   inf,   inf],
        [   0.,   -0.,   -6.,  -30.,  -90., -210.],
        [   0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.]]))

In [ ]:
  # k is single scalar, r is single 3D vector
    r.shape = (3) # FIXME do proper check
    ρ = np.linalg.norm(r)
    r̂ = r/ρ
    cosθ = r̂[2]
    sinθ = np.sqrt(1-cosθ*cosθ)
    cosφ = r̂[0]/sinθ if sinθ else 0 # avoid zero division here
    sinφ = r̂[1]/sinθ if sinθ else 1
    φ = np.arctan2(sinφ,cosφ)
    # TODO can those be views so I don't have to write the indices
    # explicitly?
    m = np.arange(nmax+1)
    n = m
    sinmφ = np.sin(m*φ)
    cosmφ = np.cos(m*φ)
    θ̂ = np.array([cosθ*cosφ,cosθ*sinφ,-sinθ])
    φ̂ = np.array([-sinφ, cosφ, 0])
    jnkr2 = sph_jn(nmax, k*ρ)
    # scipy.special.lpmn is for real arguments (see also clpmn)
    # + derivatives
    # m's are non-negative, cf. Bohren&Huffman (4.23)
    Pmn2 = lpmn(nmax, nmax, cosθ)
    
    # FIXME this is _wrong_ very near z axis – but it avoids nan 
    # and should give correct result exactly on z axis
    mdsinθ = m/(sinθ if sinθ else 1)
    
    Memn = np.outer(-mdsinθ[:,nx] * jnkr2[0][nx,:] * 
         Pmn2[0] * sinmφ[:,nx], θ̂ ) + np.outer(
         + sinθ * jnkr2[0][nx,:] * Pmn2[1] * cosmφ[:,nx], φ̂ )
    Momn = np.outer(mdsinθ[:,nx] * jnkr2[0][nx,:] * 
         Pmn2[0] * cosmφ[:,nx], θ̂ ) + np.outer(
         + sinθ * jnkr2[0][nx,:] * Pmn2[1] * sinmφ[:,nx], φ̂ )
    Npfac = jnkr2[0]/(k*ρ) + jnkr2[1]
    Nemn = np.outer(((n*(n+1)/(k*ρ))*jnkr2[0])[nx,:]
         * Pmn2[0] * cosmφ[:,nx], r̂) + np.outer( Npfac[nx,:] *
         (-sinθ) * Pmn2[1] * cosmφ[:,nx], θ̂  ) + np.outer(
         Npfac[nx,:] * (-mdsinθ)[:,nx] * Pmn2[0] * sinmφ[:,nx], φ̂ )
    Nomn = np.outer(((n*(n+1)/(k*ρ))*jnkr2[0])[nx,:]
         * Pmn2[0] * sinmφ[:,nx], r̂) + np.outer( Npfac[nx,:] *
         (-sinθ) * Pmn2[1] * sinmφ[:,nx], θ̂  ) + np.outer(
         Npfac[nx,:] * mdsinθ[:,nx] * Pmn2[0] * cosmφ[:,nx], φ̂ )
    return (Memn, Momn, Nemn, Nomn)